In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figsizeO = [8,4.7]
plt.rcParams['figure.figsize'] = figsizeO
import os
resultDir = '.' + os.path.sep + 'results' + os.path.sep

In [2]:
plotOutputPath = '.' + os.path.sep + 'plots' + os.path.sep
show = False
dpi=300

In [3]:
comparableNodes = 56 / 4
linethicknes = 1.5

In [4]:
def extractName(name):
    name = name.split(' ')[0].split(':')
    return name[1] if len(name) > 1 else name[0]

In [5]:
def extractDuration(time):
    time = time.split(' ')
    seconds = 0.0
    for t in time:
        unit = ''.join([s for s in t if s.isalpha()])
        amount = float(t[:-len(unit)])
        if unit == 'ms':
            seconds += amount / 1000
        elif unit == 's':
            seconds += amount
        elif unit == 'm':
            seconds += amount * 60
        elif unit == 'h':
            seconds += amount * 3600
        elif unit == 'd':
            seconds += amount * 86400
        else:
            raise Exception('Unknown unit', unit, time)
    return seconds

In [6]:
def extractCPU(cpu):
    return float(cpu[:-1])

In [7]:
def extractStorageInMB(storage):
    storage = storage.split(' ')
    if(len(storage) != 2):
        raise Exception('Unknown unit', storage)
    amount = float(storage[0]) 
    unit = storage[1]
    if unit == 'MB':
        return amount
    elif unit == 'GB':
        return amount * 1024
        

### Nextflow report shows:
- realtime
- peak_rss
- %CPU
- rchar
- wchar

In [8]:
aggregations = ['mean','min','max','median','std','sum','count']
byteToMb = 1024 * 1024
msPerMinute = 60 * 1000
msPerHour = msPerMinute * 60

valuesToExtract=[
    ('duration',msPerMinute,'min'),('realtime', msPerMinute,'min'),
    ('%cpu',1,'%'),('%mem', 1,'%'),
    ('rss',byteToMb,'mb'),('vmem', byteToMb,'mb'),
    ('peak_rss',byteToMb,'mb'),('peak_vmem', byteToMb,'mb'),
    ('read_bytes',byteToMb,'mb'),('write_bytes', byteToMb,'mb'),
    ('rchar',byteToMb,'mb'),('wchar', byteToMb,'mb')
                ]

In [9]:
def loadTrace(path):
    print(path)
    return pd.read_csv(path,sep=';')  

In [10]:
def reduceToMetric(trace, metric, divisor = 1):
    shortedTrace = trace[['process',metric]]
    
    l = len(shortedTrace)
    shortedTrace = shortedTrace[shortedTrace[metric] != '-']
    if l != len(shortedTrace):
        print(metric,"shortened!!!")
        shortedTrace[metric] = shortedTrace[metric].astype('float')
    
    global processes
    processes = shortedTrace['process'].unique()
    shortedTrace[metric] = shortedTrace[metric] / divisor
    result = shortedTrace.groupby('process').agg( aggregations )
    result.columns = result.columns.get_level_values(1)
    return result

def getColumn(trace, column):
    t = trace[column]
    t = t[t != '-']
    return t.astype('float')

def extractMetrics(trace):
    results = {}
    
    #stages
    preprocesses = trace[trace['process'].str.contains(':preprocess')]
    merges = trace[trace['process'].str.contains(':merge')]
    higherLevel = trace[trace['process'].str.contains('level2processing:')]
    checkResults = trace[trace['process'].str.contains('checkResults')]
    
    for v in valuesToExtract:
        results[v[0]] = reduceToMetric(trace, v[0], v[1])
        
    results['totalRuntime'] = (getColumn(checkResults,'submit').min() - getColumn(trace,'submit').min()) / msPerMinute
    results['sumRuntime'] = (getColumn(trace,'realtime').sum() - getColumn(checkResults,'realtime').sum()) / msPerHour        
    results['cpuRuntime'] = ((getColumn(trace,'realtime').multiply(getColumn(trace,'%cpu') / 100)).sum() - (getColumn(checkResults,'realtime').multiply(getColumn(checkResults,'%cpu') / 100)).sum()) / msPerHour   
    results['requestedCPURuntime'] = (getColumn(trace,'realtime').multiply(getColumn(trace,'cpus')).sum() - (getColumn(checkResults,'realtime').multiply(getColumn(checkResults,'cpus'))).sum()) / msPerHour 
    

    
    
    #preprocess
    
    results['stagePreprocess-Duration'] = (getColumn(merges,'submit').min() - getColumn(preprocesses,'submit').min()) / msPerMinute
    results['stagePreprocess-RealtimeSum'] = getColumn(preprocesses,'realtime').sum() / msPerHour    
    results['stagePreprocess-CpuRuntime'] = (getColumn(preprocesses,'realtime').multiply(getColumn(preprocesses,'%cpu') / 100)).sum() / msPerHour 
    
    #merge
    
    results['stageMerge-Duration'] = (getColumn(higherLevel,'submit').min() - getColumn(merges,'submit').min()) / msPerMinute
    results['stageMerge-RealtimeSum'] = getColumn(merges,'realtime').sum() / msPerHour
    results['stageMerge-CpuRuntime'] = (getColumn(merges,'realtime').multiply(getColumn(merges,'%cpu') / 100)).sum() / msPerHour 
    
    results['stagePreprocessMerge-Duration'] = results['stagePreprocess-Duration'] + results['stageMerge-Duration']
    
    #higher level
    
    results['stageHigherLevel-Duration'] = (getColumn(checkResults,'submit').min() - getColumn(higherLevel,'submit').min()) / msPerMinute
    results['stageHigherLevel-RealtimeSum'] = getColumn(higherLevel,'realtime').sum() / msPerHour
    results['stageHigherLevel-CpuRuntime'] = (getColumn(higherLevel,'realtime').multiply(getColumn(higherLevel,'%cpu') / 100)).sum() / msPerHour
    
    results['stageHigherLevel-rchar'] = getColumn(higherLevel,'rchar').sum() / byteToMb
    results['stageHigherLevel-wchar'] = getColumn(higherLevel,'wchar').sum() / byteToMb
        
    return results
    

In [11]:
def loadData():
    results = {}
    nodes = [x for x in os.listdir(resultDir)]
    for node in nodes:
        n = 'nf' if node == 'nf' else int(node)
        results[n] = {}
        runs = [x for x in os.listdir(resultDir + node)]
        for run in runs:
            trace = loadTrace(resultDir + node + os.path.sep + run + os.path.sep + 'trace.txt' )
            results[n][int(run)] = extractMetrics(trace)
    return results

data = loadData()
data['original'] = {
    1 : {
    'stagePreprocessMerge-Duration' : (326 * 60000 + 47 * 1000 + 913) / msPerMinute,
    'stageHigherLevel-Duration' : (29 * 60000 + 40 * 1000 + 160
    #mosaicking
    + 10 * 1000 + 903
    #pyramids
    + 46 * 1000 + 532) / msPerMinute,
    'totalRuntime' : (357 * 60000 + 33 * 1000 + 941) / msPerMinute
    }
}

.\results\1\1\trace.txt
.\results\1\2\trace.txt
.\results\1\3\trace.txt
.\results\10\1\trace.txt
.\results\10\2\trace.txt
.\results\10\3\trace.txt
.\results\14\1\trace.txt
.\results\14\2\trace.txt
.\results\14\3\trace.txt
.\results\15\1\trace.txt
.\results\15\2\trace.txt
.\results\15\3\trace.txt
.\results\20\1\trace.txt
.\results\20\2\trace.txt
.\results\20\3\trace.txt
.\results\21\1\trace.txt
.\results\21\2\trace.txt
.\results\21\3\trace.txt
.\results\3\1\trace.txt
.\results\5\1\trace.txt
.\results\5\2\trace.txt
.\results\5\3\trace.txt
.\results\nf\1\trace.txt
%cpu shortened!!!
%mem shortened!!!
rss shortened!!!
vmem shortened!!!
peak_rss shortened!!!
peak_vmem shortened!!!
read_bytes shortened!!!
write_bytes shortened!!!
rchar shortened!!!
wchar shortened!!!
.\results\nf\2\trace.txt
%cpu shortened!!!
%mem shortened!!!
rss shortened!!!
vmem shortened!!!
peak_rss shortened!!!
peak_vmem shortened!!!
read_bytes shortened!!!
write_bytes shortened!!!
rchar shortened!!!
wchar shortened!!!


In [12]:
def changeName(s):
    stage = None
    if 'preprocessmerge' in s.lower():
        stage = 'Preprocessing + Merging'
    elif 'preprocess' in s.lower():
        stage = 'Preprocessing'
    elif 'merge' in s.lower():
        stage = 'Merging'
    elif 'higherlevel' in s.lower():
        stage = 'Higher Level'
    elif 'totalruntime' in s.lower():
        stage = 'Total Runtime'
    elif 'sumruntime' in s.lower():
        stage = 'Cumulated Runtime'
    elif 'requestedcpuruntime' in s.lower():
        stage = 'Requested CPU Runtime'
    elif 'cpuruntime' in s.lower():
        stage = 'CPU Runtime'
    elif 'min' in s.lower():
        stage = 'Minimum'
    elif 'mean' in s.lower():
        stage = 'Average'
    elif 'median' in s.lower():
        stage = 'Median'
    elif 'max' in s.lower():
        stage = 'Maximum'
        
    if stage is None:
        return s
        
    workflow = None
    if s.endswith('NF'):
        workflow = 'HPS'
    elif s.endswith('Original'):
        workflow = 'HPS Original'
    
    if workflow is not None:
        return stage + ' ' + workflow
    return stage

def sortHelper(s):
    if 'preprocessmerge' in s.lower():
        return '3-' + s
    elif 'preprocess' in s.lower():
        return '1-' + s
    elif 'merge' in s.lower():
        return '2-' + s
    elif 'higherlevel' in s.lower():
        return '4-' + s
    elif 'min' in s.lower():
        return '1-' + s
    elif 'median' in s.lower():
        return '2-' + s
    elif 'mean' in s.lower():
        return '3-' + s
    elif 'max' in s.lower():
        return '4-' + s
    return '5-' + s
    
def labelLegend(labels,handles):
    labels,handles = zip(*sorted(zip(labels, handles), key=lambda t: sortHelper(t[0])))
    labels = list(labels)
    handles = list(handles)
    labels = list(map(changeName,labels))
    
    bringToEnd = ['Optimal Scaling']
    for e in bringToEnd:
        if e in labels:
            index = labels.index(e)
            l = labels.pop(index)
            labels.append(l)
            h = handles.pop(index)
            handles.append(h)
    
    return labels,handles

In [13]:
marker = {
    #'mean' : ('#696969','--','1'),'min' : ('#969696','-.','2'),'max' : ('black', ':','3'),'median': ('#d4d4d4','solid','4'), 'sum' : ('black','solid','x')
    'mean' : ('black','--','1'),'min' : ('b','-.','2'),'max' : ('g', ':','3'),'median': ('r','solid','4'), 'sum' : ('black','solid','x')
}

def rearangeDict(dictionary, fields):
    results = {}
    for run in dictionary:
        for agg in dictionary[run]:
            if agg in fields:
                for pro in dictionary[run][agg]:
                    aggC = agg
                    d = dictionary[run][aggC][pro]
                    if int(dictionary[run]['count'][pro]) == 1:
                        aggC = 'mean'
                        if agg == 'sum':
                            d = None
                    a = results.get(pro,{})
                    results[pro] = a
                    b = a.get(aggC,{})
                    a[aggC] = b 
                    b[run] = d
    return results

def plotOneProcess(dictionary, metric, process, unit, outPath = plotOutputPath, figsize = figsizeO, nCol = 1):
    plt.rcParams["figure.figsize"] = figsize
    for agg in dictionary:
        
        nfValue = None
        if 'nf' in dictionary[agg]:
            nfValue = dictionary[agg]['nf']
            del dictionary[agg]['nf']
        
        lists = sorted(dictionary[agg].items()) # sorted by key, return a list of tuples
        x, y = zip(*lists)
        
        if all(v is None for v in y):
            return
        
        plt.xticks(np.unique(x),np.array(x))
        
        plt.plot(list(map(int,x)), list(map(float,y)),label=agg, linestyle = marker[agg][1], marker = marker[agg][2], linewidth=linethicknes, c = marker[agg][0])
        
        xLabels = np.array(x)
        
        if nfValue is not None:
            plt.scatter(comparableNodes, nfValue, label = agg + 'NF', marker=marker[agg][2], linewidth=linethicknes, c = marker[agg][0])
            x = list(x)
            x.append(14)
            x.sort()
            xLabels = x.copy()
            xLabels[x.index(14)] = 'HPS'
        
    
    plt.title(metric + ' of ' + process)
    plt.xlabel('used nodes')
    plt.ylabel(metric + ' in ' + unit)
    
        
    #plt.gca().set_yticklabels(['{:.2f}%'.format(x*100) for x in plt.gca().get_yticks()]) 
    plt.xticks(np.array(x),xLabels)
    
    if len(dictionary) > 1:
        plt.legend()
        handles, labels = plt.gca().get_legend_handles_labels()
        # sort both labels and handles by labels
        labels, handles = labelLegend(labels,handles)
        plt.gca().legend(handles, labels, ncol = nCol)
        
    plt.grid(color='grey', linestyle='-', linewidth=.1)
    
    plt.savefig(outPath + process.replace(':','-') + '-' + metric + '.png', bbox_inches='tight', dpi = 300)
    plt.title(None)
    plt.savefig(outPath + process.replace(':','-') + '-' + metric + '.pdf', bbox_inches='tight', dpi = 300)
    plt.rcParams["figure.figsize"] = figsizeO
    if show:
        plt.show()
    else:
        plt.close()

def extractData(data, metric):
    allData = {}
    for d in data:
        results = []
        for run in data[d]:
            if metric not in data[d][run]:
                break
            results.append( data[d][run][metric] )
        if len(results) > 0:
            results = pd.concat(results).groupby('process').agg(['median'])   
            results.columns = results.columns.get_level_values(0)
            allData[d] = results .to_dict()
    return allData

def plot(data,metric,unit):
    allData = extractData(data,metric)
    oneExecution = rearangeDict(allData,['mean','min','max','median']) 
    sumExecution = rearangeDict(allData,['sum'])
    for proc in oneExecution:
        plotOneProcess(oneExecution[proc],metric,proc,unit)   
        if unit != '%':
            plotOneProcess(sumExecution[proc],'sum ' + metric,proc,unit)
    

In [14]:
os.makedirs( plotOutputPath + 'overleaf', exist_ok=True)

allData = extractData(data,'realtime')
oneExecution = rearangeDict(allData,['mean','min','max','median']) 
plotOneProcess(oneExecution['preprocessing:preprocess'],'realtime','preprocessing:preprocess','min', plotOutputPath + 'overleaf' + os.path.sep, figsize = (8,4), nCol = 2)  

In [15]:


for v in valuesToExtract:
    plot(data,v[0],v[2])

In [16]:
os.makedirs(plotOutputPath + 'stages', exist_ok=True)

def plotSingleMetrics(data,metric,unit,plot=True,optimumLine = False, efficiency = False, hps = False, ylim = None, outPath = plotOutputPath + 'stages'  + os.path.sep, figsize = figsizeO):
    plt.rcParams["figure.figsize"] = figsize
    allData = {}
    for nodes in data:
        results = []
        for run in data[nodes]:
            if metric in data[nodes][run]:
                results.append(data[nodes][run][metric])
        if len(results) > 0:
            allData[nodes] = np.median(results)
    print(metric,allData)
    
    originalValue = None
    if 'original' in allData:
        originalValue = allData['original']
        del allData['original']
        
    nfValue = None
    if 'nf' in allData:
        nfValue = allData['nf']
        del allData['nf']
    
    lists = sorted(allData.items()) # sorted by key, return a list of tuples
    x, y = zip(*lists)
    y0 = y[0] 
    if efficiency :
        y = (y0 / np.array(x)) / np.array(y)
    else:
        y = y / y0
        
    
    #from fractions import Fraction

    #Y = np.array([0,1/21.,1/20.,1/15.,1/10.,1/5.,1/3.,1])
    #Y_tick = np.array([])
    #for item in Y:
    #    Y_tick = np.append(Y_tick,Fraction(item).limit_denominator())
    #plt.yticks(np.unique(Y),Y_tick)
    
    if '-' in metric:
        markerTuple = markerMultiple[metric[:metric.index('-')]]
    else:
        markerTuple = ('black','solid','x','v')
    
    #plt.plot(list(map(int,x)), list(map(float,y)), label = metric,linewidth=linethicknes, marker=markerTuple[2],linestyle=markerTuple[1],color=markerTuple[0])
    plt.plot(list(map(int,x)), list(map(float,y)), label = metric,linewidth=linethicknes,linestyle=markerTuple[1],color=markerTuple[0])
    plt.title(metric)
    plt.xlabel('used nodes')
    #plt.ylabel(metric + ' in ' + unit)
    if efficiency:
        plt.ylabel('duration efficiency')
    else:
        plt.ylabel('duration in comparison to one node')
    
    xLabels = np.array(x)
    
    if originalValue is not None:
        print('original')
        xVal = (y0 / comparableNodes) / originalValue if efficiency else originalValue / y0
        plt.scatter(comparableNodes, xVal, label = metric + 'Original',linewidth=linethicknes, marker=markerTuple[3],color=markerTuple[0])
        
    if nfValue is not None:
        xVal = (y0 / comparableNodes) / nfValue if efficiency else nfValue / y0
        plt.scatter(comparableNodes, xVal, label = metric + 'NF',linewidth=linethicknes, marker=markerTuple[2],color=markerTuple[0])
        
    if hps or nfValue is not None:
        x = list(x)
        x.append(14)
        x.sort()
        xLabels = x.copy()
        xLabels[x.index(14)] = 'HPS'
        
    
    if optimumLine:
        minimum = min(list(map(int,x)))
        maximum = max(list(map(int,x))) + 1
        plt.plot(np.arange(minimum, maximum, 1), 1. / np.arange(minimum, maximum, 1), label = 'Optimal Scaling', color='red',linewidth=0.5) 
        
    #plt.hlines( 1 , 1, 21, colors='grey', linestyles='--',linewidth=0.1)
    plt.grid(color='grey', linestyle='-', linewidth=.1)
    
    plt.gca().set_yticklabels(['{:.1f}%'.format(x*100) for x in plt.gca().get_yticks()]) 
    
    
    plt.xticks(x,xLabels)
    if ylim is not None:
        plt.ylim(ylim)
    
    if plot:
        plt.legend()
        handles, labels = plt.gca().get_legend_handles_labels()
        # sort both labels and handles by labels
        labels, handles = labelLegend(labels,handles)
        plt.gca().legend(handles, labels)
        plt.savefig(outPath + metric + '.png', bbox_inches='tight', dpi = 300)
        plt.title(None)
        plt.savefig(outPath + metric + '.pdf', bbox_inches='tight', dpi = 300)
        if show:
            plt.show()
        else:
            plt.close()
            
    plt.rcParams["figure.figsize"] = figsizeO
            
def plotMultipleMetrics(metrics,endsWith,data,imageName,optimumLine = True, efficiency = False, hps = False, ylim = None, outPath = plotOutputPath + 'stages'  + os.path.sep, figsize = figsizeO, nCol = 1):
    first = optimumLine
    for sm in metrics:
        if(sm[0].endswith(endsWith)):
            plotSingleMetrics(data,sm[0],sm[1],False,first, efficiency, hps, ylim,figsize=figsize)
            first = False
    #plt.ylabel('duration in h')
    
    
    #plt.ylabel('duration in comparison to one node')
    plt.legend(loc = 'lower left' if efficiency else 'upper right')
    
    handles, labels = plt.gca().get_legend_handles_labels()
    # sort both labels and handles by labels
    labels, handles = labelLegend(labels,handles)
    plt.gca().legend(handles, labels, ncol = nCol)
    
    plt.title(imageName)
    plt.savefig(outPath + imageName + '.png', bbox_inches='tight', dpi = 300)
    plt.title(None)
    plt.savefig(outPath + imageName + '.pdf', bbox_inches='tight', dpi = 300)
    if show:
        plt.show()
    else:
        plt.close()
    

singleMetrics = [
    ('totalRuntime','min',True),
    ('sumRuntime','h',False),
    ('cpuRuntime','h',False),
    ('requestedCPURuntime','h',False),
    ('stagePreprocessMerge-Duration','min'),
    ('stageHigherLevel-Duration','min'),
    ('stageHigherLevel-RealtimeSum','h'),
    ('stageHigherLevel-CpuRuntime','h'),
    ('stagePreprocess-Duration','min'),
    ('stagePreprocess-RealtimeSum','h'),
    ('stagePreprocess-CpuRuntime','h'),
    ('stageMerge-Duration','min'),
    ('stageMerge-RealtimeSum','h'),
    ('stageMerge-CpuRuntime','h')
]  

markerMultiple = {
    #'stageMerge' : ('#696969','--','1'),'stageHigherLevel' : ('#969696','-.','2','v'),'stagePreprocess' : ('black',':','3'),'stagePreprocessMerge': ('#d4d4d4','solid','4','^')
    'stageMerge' : ('r',':','1'),'stageHigherLevel' : ('g','solid','2','v'),'stagePreprocess' : ('black','--','3'),'stagePreprocessMerge': ('b','-.','4','^')
}

for sm in singleMetrics[:4]:
    plotSingleMetrics(data,sm[0],sm[1], optimumLine=sm[2])

#plotMultipleMetrics(singleMetrics[1:1],'totalRuntime',data,'totalRuntime', True)
plotMultipleMetrics(singleMetrics[4:],'Sum',data,'stagesSumRuntime', False)
plotMultipleMetrics(singleMetrics[4:],'Duration',data,'stagesDurationEfficiency', False, True, True,(0,1.01), nCol = 2)
plotMultipleMetrics(singleMetrics[4:],'Duration',data,'stagesDuration', True, False, True)
plotMultipleMetrics(singleMetrics[4:],'CpuRuntime',data,'stagesCpuRuntime', False)


plotSingleMetrics(data,singleMetrics[0][0],singleMetrics[0][1], optimumLine=singleMetrics[0][2], outPath=plotOutputPath + 'overleaf'  + os.path.sep, figsize=(8,4))

plotMultipleMetrics(singleMetrics[4:],'Sum',data,'stagesSumRuntime', False, outPath=plotOutputPath + 'overleaf'  + os.path.sep, figsize=(8,4))

totalRuntime {1: 4882.726166666666, 10: 548.6105, 14: 431.24398333333335, 15: 412.51715, 20: 324.9487, 21: 315.35931666666664, 3: 1665.01555, 5: 1018.3698833333333, 'nf': 383.6069083333333, 'original': 357.5656833333333}
original


<ipython-input-16-6f75819bbfde>:84: UserWarning: FixedFormatter should only be used together with FixedLocator
  plt.gca().set_yticklabels(['{:.1f}%'.format(x*100) for x in plt.gca().get_yticks()])


sumRuntime {1: 241.14642083333334, 10: 244.42087916666668, 14: 245.55097027777776, 15: 246.23620805555555, 20: 247.21275166666666, 21: 248.19401833333333, 3: 241.56674361111112, 5: 241.95496916666667, 'nf': 344.13491902777776}


<ipython-input-16-6f75819bbfde>:84: UserWarning: FixedFormatter should only be used together with FixedLocator
  plt.gca().set_yticklabels(['{:.1f}%'.format(x*100) for x in plt.gca().get_yticks()])


cpuRuntime {1: 387.3936551619445, 10: 385.2889004730555, 14: 385.08604447166664, 15: 385.88310803944444, 20: 387.1791863983333, 21: 387.81550179083337, 3: 386.37261114166665, 5: 385.73817017833335, 'nf': 559.8623060616667}


...

<ipython-input-16-6f75819bbfde>:84: UserWarning: FixedFormatter should only be used together with FixedLocator
  plt.gca().set_yticklabels(['{:.1f}%'.format(x*100) for x in plt.gca().get_yticks()])


totalRuntime {1: 4882.726166666666, 10: 548.6105, 14: 431.24398333333335, 15: 412.51715, 20: 324.9487, 21: 315.35931666666664, 3: 1665.01555, 5: 1018.3698833333333, 'nf': 383.6069083333333, 'original': 357.5656833333333}
original


<ipython-input-16-6f75819bbfde>:84: UserWarning: FixedFormatter should only be used together with FixedLocator
  plt.gca().set_yticklabels(['{:.1f}%'.format(x*100) for x in plt.gca().get_yticks()])


stageHigherLevel-RealtimeSum {1: 5.371829166666667, 10: 9.570195277777778, 14: 10.547616388888889, 15: 11.111571111111111, 20: 13.183895833333333, 21: 12.763742222222222, 3: 6.221835, 5: 6.966794722222223, 'nf': 3.7667326388888887}
stagePreprocess-RealtimeSum {1: 210.74635805555556, 10: 210.49349388888888, 14: 210.42136722222222, 15: 210.98199888888888, 20: 213.16417833333333, 21: 214.13192722222223, 3: 210.5923613888889, 5: 210.40746805555557, 'nf': 307.4682319444445}
stageMerge-RealtimeSum {1: 25.080229166666665, 10: 24.610658333333333, 14: 24.615045833333333, 15: 24.129503055555556, 20: 20.824328333333334, 21: 20.919919166666666, 3: 24.738859722222223, 5: 24.58062, 'nf': 32.89519930555555}


<ipython-input-16-6f75819bbfde>:84: UserWarning: FixedFormatter should only be used together with FixedLocator
  plt.gca().set_yticklabels(['{:.1f}%'.format(x*100) for x in plt.gca().get_yticks()])


In [17]:
import shutil
os.makedirs(plotOutputPath + 'overleaf', exist_ok=True)
files = ['level2processing-processHigherLevel-%cpu']
filesStages = ['stagesDuration','stagesDurationEfficiency']
for f in files:
    shutil.copy(plotOutputPath + f + '.pdf', plotOutputPath + 'overleaf')
for f in filesStages:
    shutil.copy(plotOutputPath + 'stages' + os.path.sep + f + '.pdf', plotOutputPath + 'overleaf')

In [25]:
#Important metrics:
def getRuntime(node,metric):
    #if str(node) == '14':
    #    a = getRuntime(10,metric)
    #    b = getRuntime(15,metric)
    #    return np.interp(14, [10,15], [a,b])
        
    results = []
    for run in data[node]:
        results.append(data[node][run][metric])
    return np.median(results)

def isNumber(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
def findNodesForY(y, metric):
    nodes = []
    for n in data:
        if isNumber(n):
            nodes.append(int(n))
    nodes.sort()
    values = []
    for n in nodes:
        values.append(getRuntime(n,metric))
    index = 0
    for k in range(1, len(values)):
        if( values[ k - 1 ] >= y and values[ k ] <= y ):
            index = k
            break
    if(index == 0):
        raise Exception('No value found')
    m = ((values[k]-values[k-1])/(nodes[k]-nodes[k-1]))
    r = nodes[k-1] + (y - values[k-1]) / m
    return r

print()
print('Total Runtime')

wfDuration1Node = getRuntime(1,'totalRuntime')
wfDuration21Node = getRuntime(21,'totalRuntime')
wfDuration14Node = getRuntime(14,'totalRuntime')
wfDurationNFNode = getRuntime('nf','totalRuntime')
wfDurationOriginalNode = getRuntime('original','totalRuntime')

print('Runtime in min 1:',wfDuration1Node,'21:',wfDuration21Node,'14:',wfDuration14Node,'NF',wfDurationNFNode, 'Orig', wfDurationOriginalNode)
print('Runtime 1 -> 21',wfDuration1Node / wfDuration21Node,'x')
print('Runtime NF vs 14',1 - wfDurationNFNode / wfDuration14Node,'%')
print('Runtime Original higher 21',wfDurationOriginalNode / wfDuration21Node - 1,'%')
print('Runtime NF vs Original',wfDurationNFNode / wfDurationOriginalNode - 1,'%')

print()
print('Preprocessing')
print()

preprocessDuration1Node = getRuntime(1,'stagePreprocess-Duration')
print('stagePreprocess-Duration 1 node', preprocessDuration1Node, 'm')
preprocessDuration21Node = getRuntime(21,'stagePreprocess-Duration')
print('stagePreprocess-Duration 21 node', preprocessDuration21Node, 'm')
preprocessDuration14Node = getRuntime(14,'stagePreprocess-Duration')
print('stagePreprocess-Duration 14 node', preprocessDuration14Node, 'm')
preprocessDurationNFNode = getRuntime('nf','stagePreprocess-Duration')
print('stagePreprocess-Duration NF node', preprocessDurationNFNode, 'm')
print('stagePreprocess-Duration 1 -> 21 nodes improvement', preprocessDuration1Node / preprocessDuration21Node, 'x')
print('stagePreprocess-Duration 21 efficiency', (preprocessDuration1Node / 21) / preprocessDuration21Node, '%')

stagePreprocessRealtimeSum14Node = getRuntime(14,'stagePreprocess-RealtimeSum')
stagePreprocessRealtimeSumNFNode = getRuntime('nf','stagePreprocess-RealtimeSum')

print('stagePreprocess-RealtimeSum NF overhead', (stagePreprocessRealtimeSumNFNode / stagePreprocessRealtimeSum14Node) - 1, '%')

print()
print('Merge')
print()

mergeDuration1Node = getRuntime(1,'stageMerge-Duration')
mergeDuration21Node = getRuntime(21,'stageMerge-Duration')
mergeDuration21NodeEfficiency = ((mergeDuration1Node / 21) / mergeDuration21Node)
print('stageMerge-Duration 21 efficiency', mergeDuration21NodeEfficiency / 21 * 100, '%')

print()
print('Preproces - Merge')
print()

preprocessmergeDuration1Node = getRuntime(1,'stagePreprocessMerge-Duration')
preprocessmergeDuration14Node = getRuntime(14,'stagePreprocessMerge-Duration')
preprocessmergeDuration21Node = getRuntime(21,'stagePreprocessMerge-Duration')
preprocessmergeDurationOriginalNode = getRuntime('original','stagePreprocessMerge-Duration')
preprocessMergeDuration14NodeEfficiency = ((preprocessmergeDuration1Node / 14) / preprocessmergeDuration14Node)
preprocessMergeDurationOriginalNodeEfficiency = ((preprocessmergeDuration1Node / 14) / preprocessmergeDurationOriginalNode)
print('stagePreprocessMerge-Duration 14 efficiency', preprocessMergeDuration14NodeEfficiency * 100, '%')
print('stagePreprocessMerge-Duration Original efficiency', preprocessMergeDurationOriginalNodeEfficiency * 100, '%')
print('stagePreprocessMerge-Duration outperform original', findNodesForY(preprocessmergeDurationOriginalNode,'stagePreprocessMerge-Duration'), 'nodes')
print('stagePreprocessMerge-Duration 21 earlier than Original', (1 - preprocessmergeDuration21Node / preprocessmergeDurationOriginalNode) * 100, '%')

print()
print('Higher Level')
print()

higherLevelDuration1Node = getRuntime(1,'stageHigherLevel-Duration')
print('higherLevelDuration1Node-Duration 1 node', higherLevelDuration1Node, 'min')

higherLevelDuration14Node = getRuntime(14,'stageHigherLevel-Duration')
higherLevelDuration21Node = getRuntime(21,'stageHigherLevel-Duration')
higherLevelDurationNFNode = getRuntime('nf','stageHigherLevel-Duration')
higherLevelDurationOriginalNode = getRuntime('original','stageHigherLevel-Duration')

print('stageHigherLevel runtime with 1 nodes higher 21 node', higherLevelDuration1Node / higherLevelDuration21Node, 'x')
higherLevelDuration21NodeEfficiency = ((higherLevelDuration1Node / 21) / higherLevelDuration21Node)
print('stagePreprocessMerge-Duration 21 efficiency', higherLevelDuration21NodeEfficiency * 100, '%')



higherLevelRChar1Node = getRuntime(1,'stageHigherLevel-rchar') / 1024
higherLevelWChar1Node = getRuntime(1,'stageHigherLevel-wchar') / 1024
print('Higher Level I/O: r:',higherLevelRChar1Node,'gb','w:', higherLevelWChar1Node,'gb')


higherLevelRealtimeSum1Node = getRuntime(1,'stageHigherLevel-RealtimeSum')
higherLevelRealtimeSum20Node = getRuntime(20,'stageHigherLevel-RealtimeSum')

print('stagePreprocessMerge-RuntimeSum Increase 1->20', higherLevelRealtimeSum20Node / higherLevelRealtimeSum1Node * 100, '%')

higherLevelDuration14NodeEfficiency = ((higherLevelDuration1Node / 14) / higherLevelDuration14Node)
higherLevelDurationOriginalNodeEfficiency = ((higherLevelDuration1Node / 14) / higherLevelDurationOriginalNode)
higherLevelDurationNFNodeEfficiency = ((higherLevelDuration1Node / 14) / higherLevelDurationNFNode)
print('higherLevelMerge-Duration 14 efficiency', higherLevelDuration14NodeEfficiency * 100, '%')
print('higherLevelMerge-Duration Original efficiency', higherLevelDurationOriginalNodeEfficiency * 100, '%')
print('higherLevelMerge-Duration NF efficiency', higherLevelDurationNFNodeEfficiency * 100, '%')
print('stageHigherLevel runtime with 21 nodes vs Original node', higherLevelDuration21Node / higherLevelDurationOriginalNode, '%')

print()
print('Discussion')
print()


totalDuration21NodeEfficiency = ((wfDuration1Node / 21) / wfDuration21Node)
print('21 Node total efficiency', totalDuration21NodeEfficiency * 100, '%')
print('21 Node preprocessing share', preprocessDuration21Node / wfDuration21Node * 100, '%')
higherLevelDuration21NodeEfficiency = ((higherLevelDuration1Node / 21) / higherLevelDuration21Node)
print('higherLevelMerge-Duration 21 efficiency', higherLevelDuration21NodeEfficiency * 100, '%')


print()
print('Conclusion')
print()

print('total runtime outperform original', findNodesForY(preprocessmergeDurationOriginalNode,'totalRuntime'), 'nodes')


Total Runtime
Runtime in min 1: 4882.726166666666 21: 315.35931666666664 14: 431.24398333333335 NF 383.6069083333333 Orig 357.5656833333333
Runtime 1 -> 21 15.483056655109655 x
Runtime NF vs 14 0.11046432377278781 %
Runtime Original higher 21 0.13383580073925194 %
Runtime NF vs Original 0.0728292065313314 %

Preprocessing

stagePreprocess-Duration 1 node 4338.341333333334 m
stagePreprocess-Duration 21 node 225.42235 m
stagePreprocess-Duration 14 node 338.65245 m
stagePreprocess-Duration NF node 336.30813333333333 m
stagePreprocess-Duration 1 -> 21 nodes improvement 19.245391299191645 x
stagePreprocess-Duration 21 efficiency 0.9164472047234117 %
stagePreprocess-RealtimeSum NF overhead 0.4612025195128251 %

Merge

stageMerge-Duration 21 efficiency 2.4981005038300754 %

Preproces - Merge

stagePreprocessMerge-Duration 14 efficiency 90.01260345244032 %
stagePreprocessMerge-Duration Original efficiency 99.75581323432317 %
stagePreprocessMerge-Duration outperform original 15.782133764103762